## 3. Jupiter Notebook zum NerDH Tutorial


[Zurück zum Web-Tutorial](https://easyh.github.io/NerDH/tut/)

### 3.1 Trainings-, Validierungs- und Testdaten

Beim maschinellen Lernen benötigen wir einen Trainingsdatensatz, um ein Modell richtig zu trainieren und einen Testdatensatz, um das Modell zu bewerten.  Der Datensatz [`taggedData.json`](https://github.com/easyh/NerDH/blob/main/data/datensets/taggedData.json) umfasst den komplett annotierten Text [**München 1611**](https://hainhofer.hab.de/reiseberichte/muenchen1611?v={%22view%22:%22info%22}). Bei **unüberwachten Lernmethoden** wird dieser Datensatz in der Regel in mindestens drei verschiedene Datensätze unterteilt: **Training-, Validierung- und Testdaten**. In unserem Fall übernehmen die Trainingsdaten ([`testData.json`](https://github.com/easyh/NerDH/blob/main/data/datensets/taggedData.json)) bei uns die annotierten Daten aus dem Text [**München 1603**](https://hainhofer.hab.de/reiseberichte/muenchen1603). Die Aufteilung unserer Daten sieht wie folgt aus:
<div>
<img src="../nerdh_tutorial/docs/img/datenset_lg.png" width="400"/>
</div>



Um jetzt unseren großen Datensatz [`taggedData.json`](https://github.com/easyh/NerDH/blob/main/data/datensets/taggedData.json) in zwei Datensets aufzuteilen, lesen wir diesen zunächst ein und speichern nur die Einträge von  `annotations` in der Variablen `TAGGED_DATA`, damit wir die Einträge zählen können. Danach ermitteln wir die Grenze (80:20), damit wir den ursprünglichen Datensatz in kleinere Datensätze zu je 80% und 20% aufteilen können.

In [ ]:
import json

f = open('../data/datensets/taggedData.json')
data = json.load(f)
TAGGED_DATA = data['annotations']

print(len(TAGGED_DATA)*0.8)

Bevor wir den Datensatz an den ermittelten Grenzen in die drei Datensets aufteilen, mischen wir die Einträge noch einmal durch, damit die Verteilung zufällig ist. Hier lassen wir uns dann jeweils die Länge ausgeben, um das Ergebnis zu überprüfen. Zusätzlich lassen wir uns auch noch die Größe des Testdatensatz ausgeben, um zu überprüfen, ob die 70:20:10 Verteilung gewährleistet ist.

In [ ]:
import random 

random.shuffle(TAGGED_DATA)
train_data = TAGGED_DATA[:697]
val_data = TAGGED_DATA[697:]

print("Trainingsdaten: " + str(len(train_data)))
print("Validierungsdaten: " + str(len(val_data)))

#Zum vergleich, lassen wir uns auch die Göße von unseren Testdaten ausgeben
f = open('../data/datensets/testData.json')
data = json.load(f)
test_data = data['annotations']
print("Testdaten: " + str(len(test_data)))


Anschließend speichern wir die Datensets im JSON-Format ab.

In [ ]:
with open ('../data/datensets/trainData.json', 'w', encoding='utf-8') as train: 
    json.dump(train_data, train, ensure_ascii=False, indent=4)
with open ('../data/datensets/valuationData.json', 'w', encoding='utf-8') as val: 
    json.dump(val_data, val, ensure_ascii=False, indent=4)

Da wir weiter oben nur die Einträge aus der Klasse  `annotations` der JSON-Datei übernommen haben, müssen wir jetzt noch einmal manuell bei `trainData.json` sowie `validationData.json` die `classes` hinzufügen, damit unsere Kategorien für die Entitäten nicht verloren gehen. Dazu setzen wir an den Anfang des Dokuments folgendes und ans Ende eine `}` um das Dokument zu schließen.

```json
{"classes": ["PERSON","ORT", "ORGANISATION","OBJEKT","ZEIT"],
    "annotations": 
```

<br>

---

Jetzt müssen die Datensets im `JSON`-Format nur noch ins `spaCy`-Format konvertiert werden. Dafür importieren wir zunächst die entsprechenden Bibliotheken und das mittlere Sprachmodell von `spaCy`.

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

#neues spacy Model laden
nlp = spacy.load("de_core_news_md") 

#DocBin Objekt erstellen
db = DocBin() 

Jetzt müssen wir für jedes Datenset nur noch folgenden Code ausführen, damit die Datensets im `spaCy`-Datenformat sind. 
Der Code ist von [hier](https://towardsdatascience.com/using-spacy-3-0-to-build-a-custom-ner-model-c9256bea098)

In [ ]:
#Trainingsdaten
f = open('../data/datensets/trainData.json')
TRAIN_DATA = json.load(f)

for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

#Docbin Objekt speichern
db.to_disk("../data/datensets/trainData.spacy") 

In [ ]:
#Valuiierungsdaten
f = open('../data/datensets/valuationData.json')
VAL_DATA = json.load(f)

for text, annot in tqdm(VAL_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

#Docbin Objekt speichern
db.to_disk("../data/datensets/valuationData.spacy") 

In [ ]:
#Testdata
f = open('../data/datensets/testData.json')
TEST_DATA = json.load(f)

for text, annot in tqdm(TEST_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

#Docbin Objekt speichern
db.to_disk("../data/datensets/testData.spacy")

Damit ist das Preprocessing abgeschlossen und wir können mit dem Training des Modells beginnen.

Im Github Repository befinden sich die Datensets im `JSON`- & `spaCy`-Format [**hier**](https://github.com/easyh/NerDH/blob/main/data/datensets) (`data/datensets`).